## Exercise Set 1 for OSM 

### Dynamic Programming with John Stachurski

### Solutions by Wei Han Chia

This notebook contains the first set of exercises for the [OSM](https://bfi.uchicago.edu/osm) bootcamp dynamic programming section.

We will use the following libraries:

In [1]:
import numpy as np
import matplotlib.pyplot as plt

### Exercise 1

Consider the data

In [4]:
A = [[0.6, 0.1, -0.3],
     [0.5, -0.4, 0.2],
     [1.0, -0.2, 1.1]]

b = [[12],
     [10],
     [-1]]

A, b = map(np.asarray, (A, b))

The equation $x = A x + b$ has a unique solution in $\mathbb R^3$.  Show that this is true by checking the spectral radius condition.

In [3]:
from scipy.linalg import eigvals, solve
evs = eigvals(A)
ρ = max(abs(evs))
print(ρ)

0.965538166352


Compute the solution using both successive approximations --- that is,

$$ x_{t+1} = A x_t + b $$

and matrix algebra.

In [7]:
# Solving with Successive Approximations

error = 1
epsilon = 1e-12
x_int = np.asarray([[0],[0],[0]])

while error > epsilon:
    x_new = np.dot(A,x_int) + b
    error = np.linalg.norm(x_new - x_int)
    x_int = x_new

print(x_new)

[[ -0.89552239]
 [ 13.34328358]
 [ 45.64179104]]


In [10]:
# Solving with Linear Algebra

I_3 = np.eye(3)

ans = np.dot(np.linalg.inv(I_3 - A),b)
print(ans)

[[ -0.89552239]
 [ 13.34328358]
 [ 45.64179104]]


### Exercise 2

In a standard job search model, an agent 


* receives offer $w_t$ in period $t$ where $\{w_t\}$ is  iid
* accepts this offer at time $t$ or remains unemployed
    *  if remains unemployed, then receives compensation $c > 0$
    * if accepts, then works indefinitely at this wage

The agent discounts the future at rate $\beta \in (0, 1)$


It can be shown that the following is an optimal strategy: 

* Set a reservation wage $\bar w$
* Accept the first offer $w_t$ such that $w_t \geq \bar w$


It can also be shown (details omitted) that $\bar w$ should satisfy

$$
    \bar w
    = c (1-\beta) + \beta
    \sum_{k=1}^K \max \left\{
        w_k ,\, \bar w
    \right\}
    \, p_k
$$

Here 

* $w_1, \ldots, w_K$ are the possible wage values, occuring with probabilities $p_1, \ldots, p_K$
* $c > 0$

Does there always exists a $x \in [0, \infty)$ that solves the equation
$$
    x
    = c (1-\beta) + \beta
    \sum_{k=1}^K \max \left\{
        w_k ,\, x
    \right\}
    \, p_k
$$
Is it unique?  Suggest a strategy for computing it

#### Hints

There are different ways to show this, but I found the following facts useful:

*  If $x_1, \ldots, x_K$ are any $K$ numbers, then

$$ \left| \sum_{k=1}^K x_k \right| \leq \sum_{k=1}^K |x_k| $$

* For any $a, x, y$ in $\mathbb R$, 
    
$$ 
    \left| 
            \max \left\{ a,\, x \right\} - \max \left\{ a,\, y \right\} 
        \right|
        \leq | x - y |
$$


You can convince yourself of the second inequality by sketching and checking different cases...

We will use the facts above. To see that they are true, note that the first is just repeated application of the triangle inequality. For the second equality, let us prove it by cases.

If $x, y < a$, then the inequality holds trivially, since $0 \leq |x-y|$.

If $x > a, y <a$, then the inequality holds since $|x - a| \leq |x -y|$ when $y < a$.

If $x < a, y > a$, then the inequality holds since $|a - y| \leq |x - y|$ when $x < a$.

Finally if $x, y > a$, then the inequality holds trivially since $|x - y| = |x - y|$.

Now in order for prove that there exists a unique solution, we will apply the Banach Contraction mapping theorem to find a fixed point $x$. We can define the mapping $T$ as:
$$ T(x) = c(1 - \beta) + \beta \sum_{k=1}^{K} \max \{ w_k, x\} p_k $$

Now we want to prove that $T$ is a contraction mapping. To do this, consider $|T(x) - T(x')|$.
$$ \begin{align}
\left| T(x) - T(x')\right| &= \left| c(1-\beta) + \beta \sum_{k=1}^{K} \max \{ w_k, x\} p_k - c(1-\beta) - \beta \sum_{k=1}^{K} \max \{ w_k, x'\} p_k \right| \\
&= \left| \beta \sum_{k=1}^{K} (\max\{w_k, x\} - \max\{w_k - x'\})p_k \right| \\
& \leq  \beta \sum_{k=1}^{K} \left|(\max\{w_k, x\} - \max\{w_k - x'\})p_k \right| \\
& \leq \beta |x - x'| \sum_{k=1}^{K} p_k \\
& = \beta |x - x'|
\end{align}$$

Now since $\beta \in (0,1)$, it follows that $T$ is a contraction mapping. Now simply applying the contraction mapping theorem would prove that there is a unique solution. Further, we can compute this solution by using the same successive approximation method as in question 1.

Note here that we have also used the fact that $[0,\infty)$ is a complete metric space. 


### Exercise 3

Let 

* $\beta = 0.96$
* $(w_1, w_2, w_3) = (0.5, 1.0, 1.5)$
* $(p_1, p_2, p_3) = (0.2, 0.4, 0.4)$

Compute and plot the reservation wage for unemployment compensation values

In [85]:
c_vals = np.linspace(1, 2, 100)
wvec = np.array([[0.5],[1.0],[1.5]])
p = np.array([[0.2],[0.4],[0.4]])

wbar = np.ones((100))
wbarnew = np.ones((100))
error = np.ones((100))
epsilon = 1e-100
beta = 0.96


for i in range(0,100):
    while error[i] > epsilon:
        wtemp = wbar[i]*np.ones((3,1))
        wbarnew[i] = c_vals[i]*(1 - beta) + beta*np.sum(np.maximum(wvec, wtemp)*p)
        error[i] = abs(wbar[i] - wbarnew[i])
        wbar[i] = wbarnew[i]

wbar

array([ 1.45283019,  1.45378311,  1.45473604,  1.45568897,  1.45664189,
        1.45759482,  1.45854774,  1.45950067,  1.46045359,  1.46140652,
        1.46235944,  1.46331237,  1.46426529,  1.46521822,  1.46617115,
        1.46712407,  1.468077  ,  1.46902992,  1.46998285,  1.47093577,
        1.4718887 ,  1.47284162,  1.47379455,  1.47474747,  1.4757004 ,
        1.47665333,  1.47760625,  1.47855918,  1.4795121 ,  1.48046503,
        1.48141795,  1.48237088,  1.4833238 ,  1.48427673,  1.48522966,
        1.48618258,  1.48713551,  1.48808843,  1.48904136,  1.48999428,
        1.49094721,  1.49190013,  1.49285306,  1.49380598,  1.49475891,
        1.49571184,  1.49666476,  1.49761769,  1.49857061,  1.49952354,
        1.50505051,  1.51515152,  1.52525253,  1.53535354,  1.54545455,
        1.55555556,  1.56565657,  1.57575758,  1.58585859,  1.5959596 ,
        1.60606061,  1.61616162,  1.62626263,  1.63636364,  1.64646465,
        1.65656566,  1.66666667,  1.67676768,  1.68686869,  1.69

Is the reservation wage increasing or decreasing in this value?  Does that coincide with your intuition?

The reservation wage is increasing in consumption. This does indeed coincide with our intuition, since a higher unemployment benefit would mean that individuals would need to be provided a higher wage to decide to start working. 